<a href="https://colab.research.google.com/github/Someoneamm/learn-git/blob/main/Youtube_ANN_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project = "gee-ann")

In [ ]:
#Importing TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
import numpy as np

tf.__version__

'2.15.0'

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
#GCS Configuration
PROJECT = 'gee-ann'
REGION = 'us-east1'

# Some Information about training data
FEATURE_NAMES = ['B4','B3','B2','NDVI','LC']
BANDS = ['B4','B3','B2','NDVI']
LABEL = 'LC'
N_CLASSES = 4 #Number of Classes

# Imports (Training and Testing Data)
TRAIN_FILE_PATH = 'gs://gee_dl_2/isb_training_points.tfrecord.gz'
TEST_FILE_PATH = 'gs://gee_dl_2/isb_testing_points.tfrecord.gz'

# (About Model that will later be exported)
MODEL_DIR = 'gs://gee_dl_2/Model_ANN'
MODEL_NAME = 'GEE_TF_ANN_classification'
VERSION_NAME = 'V0' #Optional

# Exported Image Details
EXPORTED_IMAGE_PREFIX = 'Islamabad_Image' #exported image from GEE JS API to GCS **Important**
OUTPUT_IMAGE_FILE = 'gs://gee_dl_2/classified_image_ANN.TFRecord'
OUTPUT_ASSET_ID = 'projects/gee-ann/assets/classified_image_ANN'

In [ ]:
# Create a dataset from the TFRecord file in Cloud Storage. (Prepare for TensorFlow)
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
# Print the first record to check.
iter(train_dataset).next()

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\x85\x01\n\x0e\n\x02B2\x12\x08\x12\x06\n\x04\xdd\xb5\x84=\n\x12\n\x06random\x12\x08\x12\x06\n\x04\xe5\x18\x16>\n\x0e\n\x02B3\x12\x08\x12\x06\n\x04\x12\xa5\xbd=\n\x0e\n\x02B4\x12\x08\x12\x06\n\x04\xa7y\xc7=\n\x0e\n\x02LC\x12\x08\x12\x06\n\x04\x00\x00\x00\x00\n\x10\n\x04NDVI\x12\x08\x12\x06\n\x04\x83\x15\xfa>\n\x1d\n\x0csystem:index\x12\r\n\x0b\n\t1_1_1_0_0'>

In [ ]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]


# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))

def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
iter(parsed_dataset).next()

({'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0648], dtype=float32)>,
  'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0926], dtype=float32)>,
  'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0974], dtype=float32)>,
  'NDVI': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.48844537], dtype=float32)>},
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)

In [ ]:
# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))

# Map the to_tuple function
input_dataset = parsed_dataset.map(to_tuple)

print(input_dataset,'Printed \n \n')
iter(input_dataset).next()

<_MapDataset element_spec=(TensorSpec(shape=(1, 4), dtype=tf.float32, name=None), TensorSpec(shape=(1, 4), dtype=tf.float32, name=None))> Printed 
 



(<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.0648    , 0.0926    , 0.0974    , 0.48844537]], dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[1., 0., 0., 0.]], dtype=float32)>)

In [ ]:
# Define the layers in the model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

In [ ]:
# Compile the model with the specified loss function.
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
input_dataset = input_dataset.shuffle(64).batch(8)
iter(input_dataset).next() #printing the first training record just for check

(<tf.Tensor: shape=(8, 1, 4), dtype=float32, numpy=
 array([[[ 0.0408    ,  0.0607    ,  0.0568    ,  0.5534591 ]],
 
        [[ 0.0316    ,  0.0634    ,  0.0362    ,  0.76455283]],
 
        [[ 0.0421    ,  0.0662    ,  0.0706    ,  0.5394651 ]],
 
        [[ 0.0247    ,  0.0474    ,  0.0396    ,  0.48370275]],
 
        [[ 0.0442    ,  0.0692    ,  0.0662    ,  0.5909793 ]],
 
        [[ 0.0331    ,  0.0631    ,  0.0364    , -0.43307087]],
 
        [[ 0.0212    ,  0.0416    ,  0.0318    ,  0.7805383 ]],
 
        [[ 0.0315    ,  0.0569    ,  0.0439    ,  0.6845131 ]]],
       dtype=float32)>,
 <tf.Tensor: shape=(8, 1, 4), dtype=float32, numpy=
 array([[[1., 0., 0., 0.]],
 
        [[1., 0., 0., 0.]],
 
        [[1., 0., 0., 0.]],
 
        [[0., 1., 0., 0.]],
 
        [[1., 0., 0., 0.]],
 
        [[0., 1., 0., 0.]],
 
        [[1., 0., 0., 0.]],
 
        [[1., 0., 0., 0.]]], dtype=float32)>)

In [ ]:
# Fitting the model
model.fit(x=input_dataset, epochs=100)


Epoch 1/100
23/23 [==============================] - 1s 4ms/step - loss: 1.4056 - accuracy: 0.1538
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 1.3662 - accuracy: 0.3462
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 1.3300 - accuracy: 0.4451
Epoch 4/100
23/23 [==============================] - 0s 4ms/step - loss: 1.2932 - accuracy: 0.5440
Epoch 5/100
23/23 [==============================] - 0s 4ms/step - loss: 1.2685 - accuracy: 0.5275
Epoch 6/100
23/23 [==============================] - 0s 4ms/step - loss: 1.2360 - accuracy: 0.5714
Epoch 7/100
23/23 [==============================] - 0s 4ms/step - loss: 1.2060 - accuracy: 0.6044
Epoch 8/100
23/23 [==============================] - 0s 4ms/step - loss: 1.1763 - accuracy: 0.6099
Epoch 9/100
23/23 [==============================] - 0s 4ms/step - loss: 1.1343 - accuracy: 0.6099
Epoch 10/100
23/23 [==============================] - 0s 4ms/step - loss: 1.0944 - accuracy: 0.6264
Epoch 11/

In [ ]:
# Model Summary and Saving the model
model.summary()
model.save(MODEL_DIR, save_format='tf')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1, 64)             320       
                                                                 
 dropout_2 (Dropout)         (None, 1, 64)             0         
                                                                 
 dense_5 (Dense)             (None, 1, 4)              260       
                                                                 
Total params: 580 (2.27 KB)
Trainable params: 580 (2.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# CHECK THE MODEL accuracy on the validation dataset
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(to_tuple)
    .batch(2))

model.evaluate(test_dataset)

19/19 [==============================] - 1s 4ms/step - loss: 0.5222 - accuracy: 0.7368


[0.5222229957580566, 0.7368420958518982]

In [ ]:
# Classifying The Image
# We will now use the DNN model to predict the output class for all pixels of the input image.
# The input composite for the entire basin has been exported to GCS. We will read the image and run prediction for each image path.
# When you export an image from Earth Engine as TFRecords, you get 2 files
# .tfrecord.gz files containing image patches
# mixer.json file containing image metadata and georeferencing information

# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://gee_dl_2'

# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if EXPORTED_IMAGE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

print(image_files_list)

['gs://gee_dl_2/Islamabad_Image00000.tfrecord.gz', 'gs://gee_dl_2/Islamabad_Image00001.tfrecord.gz', 'gs://gee_dl_2/Islamabad_Image00002.tfrecord.gz', 'gs://gee_dl_2/Islamabad_Image00003.tfrecord.gz']


In [ ]:
# Make sure the files (patches) are in the right order.
image_files_list.sort()
image_files_list

['gs://gee_dl_2/Islamabad_Image00000.tfrecord.gz',
 'gs://gee_dl_2/Islamabad_Image00001.tfrecord.gz',
 'gs://gee_dl_2/Islamabad_Image00002.tfrecord.gz',
 'gs://gee_dl_2/Islamabad_Image00003.tfrecord.gz']

In [ ]:
import json
# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
mixer

{'projection': {'crs': 'EPSG:4326',
  'affine': {'doubleMatrix': [8.983152841195215e-05,
    0.0,
    72.81384366959193,
    0.0,
    -8.983152841195215e-05,
    33.811958473559905]}},
 'patchDimensions': [32, 32],
 'patchesPerRow': 200,
 'totalPatches': 22400}

In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32)
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

In [ ]:
# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)
iter(image_dataset).next() #Display first record from batch

(<tf.Tensor: shape=(1024, 1, 4), dtype=float32, numpy=
 array([[[0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.]]], dtype=float32)>,)

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(image_dataset, steps=patches, verbose=1)

22400/22400 [==============================] - 817s 36ms/step


In [ ]:
# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])
print(predictions[0].shape)

[[1.0879214e-03 9.9119556e-01 6.9950796e-03 7.2146673e-04]]
(1, 4)


In [ ]:
# Instantiate the writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[]]
cur_patch = 1
for prediction in predictions:
  patch[0].append(int(tf.argmax(prediction, 1)))

  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == patch_width * patch_height):
    print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(value=patch[0]))
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[]]
    cur_patch += 1

writer.close()

Streaming output truncated to the last 5000 lines.
Done with patch 17401 of 22400...
Done with patch 17402 of 22400...
Done with patch 17403 of 22400...
Done with patch 17404 of 22400...
Done with patch 17405 of 22400...
Done with patch 17406 of 22400...
Done with patch 17407 of 22400...
Done with patch 17408 of 22400...
Done with patch 17409 of 22400...
Done with patch 17410 of 22400...
Done with patch 17411 of 22400...
Done with patch 17412 of 22400...
Done with patch 17413 of 22400...
Done with patch 17414 of 22400...
Done with patch 17415 of 22400...
Done with patch 17416 of 22400...
Done with patch 17417 of 22400...
Done with patch 17418 of 22400...
Done with patch 17419 of 22400...
Done with patch 17420 of 22400...
Done with patch 17421 of 22400...
Done with patch 17422 of 22400...
Done with patch 17423 of 22400...
Done with patch 17424 of 22400...
Done with patch 17425 of 22400...
Done with patch 17426 of 22400...
Done with patch 17427 of 22400...
Done with patch 17428 of 22400.

In [ ]:
#Upload back to GEE
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

Started upload task with ID: YAEOSN45WWQDTYVHLOUBCESX
